In [3]:
% pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io.wavfile import read as sciread
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [4]:
def calcFFT(audio, fs):
    transformed = np.fft.fft(audio)
    f = np.arange(0, len(transformed))*fs/len(transformed)
    f = f[:44100]
    transformed = transformed[:44100]
    return (f, transformed)

In [5]:
def plotFFT(audio, fs):
    f, transformed = calcFFT(audio, fs)
    plt.plot(f, np.abs(transformed))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.show()

In [6]:
def get_info(fpath, label):
    signal = librosa.core.load(fpath, 44100)[0]
    f, transformed = calcFFT(signal, 44100)
    transformed = np.array(np.abs(transformed))
    data = {}
    data['fpath'] = fpath
    data['label'] = label
    data['median'] = np.median(transformed)
    data['mean'] = np.mean(transformed)
    data['1Q'] = np.percentile(transformed, 25)
    data['2Q'] = np.percentile(transformed, 75)
    data['IQR'] = stats.iqr(transformed)
    data['min'] = np.min(transformed)
    data['max'] = np.max(transformed)
    data['std'] = np.std(transformed)
    data['skew'] = stats.skew(transformed, bias=True)
    return data

In [7]:
train = pd.read_csv('train.csv')
train.head(10)
labels = list(train.label.unique())

In [8]:
files = []
for i in range(len(train)):
    file = train.loc[i].fname
    files.append(f'audio_train/audio_train/{file}')

In [9]:
train['fpath'] = files

In [10]:
train.head()

,fname,label,manually_verified,fpath
0,00044347.wav,Hi-hat,0,audio_train/audio_train/00044347.wav
1,001ca53d.wav,Saxophone,1,audio_train/audio_train/001ca53d.wav
2,002d256b.wav,Trumpet,0,audio_train/audio_train/002d256b.wav
3,0033e230.wav,Glockenspiel,1,audio_train/audio_train/0033e230.wav
4,00353774.wav,Cello,1,audio_train/audio_train/00353774.wav


In [21]:
train.to_csv(path_or_buf='train.csv')

In [11]:
working = pd.DataFrame()

In [32]:
for i in range(5000, len(train.fname)):
    fpath = train.loc[i, 'fpath']
    label = train.loc[i, 'label']
    info = get_info(fpath, label)
    df = pd.DataFrame(info, index=[i])
    working = pd.concat([working, df])

In [33]:
working.to_csv(path_or_buf='working.csv')